In [598]:
# !pip install pycaret


In [599]:
# pip install markupsafe==2.0.1

In [775]:
import sklearn
import pandas as pd
import matplotlib.pyplot as  plt
import numpy as np 
import seaborn as sns

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup , compare_models
from sklearn.model_selection import KFold, cross_val_score


In [776]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
pd.set_option('max_rows',100)
pd.set_option('max_columns',100)



# Data Cleaning

In [777]:
target = train['SalePrice']
train.drop(['Id', 'SalePrice'] , axis =1,inplace = True)
test.drop('Id', axis = 1, inplace = True)
cdata = pd.concat([train,test], axis = 0).reset_index(drop = True)


ensuring data types

In [778]:

cdata['MSSubClass'] = cdata['MSSubClass'].astype(str)
#since numeric data nominal data 

In [779]:

categorical_features = cdata.select_dtypes(object).columns
numeric_features = cdata.select_dtypes(np.number).columns
numeric_df = cdata.select_dtypes(np.number)



Filling missing categorical values 

In [780]:
#NA means something and not missing value 

misc_ele = [
    'Alley', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1',
    'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'
]
for column in misc_ele:
    cdata[column] = cdata[column].fillna("None")

In [781]:
mode_feat = categorical_features.drop(misc_ele)
for column in mode_feat:
  cdata[column] = cdata[column].fillna(cdata[column].mode()[0])


filling missing numeric data

In [782]:

def knn_impute(df, na_target):
  #imputes using knn(avg of all the nearest datapoints)  data must contain atleast one non na column
  df = df.copy()

  numeric_df = df.select_dtypes(np.number)
  non_na_columns = numeric_df.loc[:, numeric_df.isna().sum() == 0].columns

  y_train = numeric_df.loc[numeric_df[na_target].isna() == False, na_target]
  x_train =numeric_df.loc[numeric_df[na_target].isna() == False, non_na_columns]
  x_test = numeric_df.loc[numeric_df[na_target].isna() == True, non_na_columns]

  knn = KNeighborsRegressor()
  knn.fit(x_train, y_train)

  y_pred = knn.predict(x_test)

  df.loc[df[na_target].isna() == True, na_target] = y_pred

  return df


for column in numeric_df.loc[:,numeric_df.isna().sum() > 0].columns:

  cdata = knn_impute(cdata, column)




In [783]:
# def drop_fieldna(df):
#   for field in df:
#     if df[field].isnull().sum()/df.shape[0] > 0.49:
#       df.drop(field, axis = 1, inplace=True)
# drop_fieldna(cdata)
# df.drop('Id', axis = 1, inplace = True)


#Feature Engineering

In [784]:
# cor = cdata.corr()


# for i in cor.columns:
#   for j in cor[i].index:
#     if cor.loc[j,i] > 0.6 and i!=j:
#       print(j,i,cor.loc[j,i])

In [785]:
    
    cdata['GarageYrBltn'] = abs(cdata['YrSold'] - cdata['GarageYrBlt'])
    cdata['YearRemodAddn'] = abs(cdata['YrSold'] - cdata['YearRemodAdd'])
    cdata['YearBuiltn'] = abs(cdata['YrSold'] - cdata['YearBuilt'])
    cdata['SF'] = cdata['1stFlrSF']+cdata['2ndFlrSF']+cdata['TotalBsmtSF']+cdata['GrLivArea']+cdata['HalfBath']+cdata['FullBath']
   
    # for i in ['1stFlrSF','2ndFlrSF','TotalBsmtSF','GrLivArea','HalfBath','FullBath','GarageYrBltn','YearRemodAdd','YearBuilt']:
    #   cdata.drop(i,axis= 1, inplace = True)


#feature Transformation


In [786]:
def skew_log_transform(df):
  # using log1p transform to fix skew in data
  df = df.copy()
  numeric_features = df.select_dtypes(np.number).columns
  for column in numeric_features:
    skew = abs(scipy.stats.skew(df[column]))
  
    if skew >= 0.5:
      df[column] = np.log1p(df[column])
      # l1p = np.log1p(df[column])
   
      # tskew = abs(scipy.stats.skew(l1p))
      # # print(tskew)
      # # print(l1p)
      # if tskew < skew:
      #   df[column] = l1p
  return df

cdata = skew_log_transform(cdata)


In [787]:
cdata['MoSold'] = (-np.cos(0.5236 * cdata['MoSold']))


one hot encoding

In [788]:

cdata = pd.get_dummies(cdata,drop_first= True)






Scaling

In [789]:
scaler = StandardScaler()
scaler.fit(cdata)
cdata = pd.DataFrame(scaler.transform(cdata), index = cdata.index, columns = cdata.columns)

target transformation

In [790]:
log_target = np.log(target)

#model selection

In [791]:

# final = cdata.loc[:,~final.columns.duplicated()]

df_train = cdata.iloc[:1460,:]
df_test = cdata.iloc[1460:,:]
final = pd.concat([df_train,log_target], axis = 1)

In [792]:

setup(final,target='SalePrice')

,Description,Value
0,session_id,8696
1,Target,SalePrice
2,Original Data,"(1460, 278)"
3,Missing Values,False
4,Numeric Features,48
5,Categorical Features,229
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1021, 266)"


(False,
 False,
 False,
 'lightgbm',
 False,
 False,
 'SalePrice',
       LotFrontage   LotArea  OverallQual  OverallCond  YearBuilt  MasVnrArea  \
 240      0.349325  0.019960     1.355551    -0.435439   1.202131    0.572454   
 454     -0.154368  0.083640    -0.772552    -0.435439   0.161526   -0.804554   
 385     -1.252596 -2.019119     1.355551    -0.435439   1.072966    0.275880   
 95       0.223241  0.179968    -0.063185     1.906370   0.716427    0.810105   
 1130    -0.064176 -0.259704    -1.481920    -2.777249  -1.431397   -0.804554   
 ...           ...       ...          ...          ...        ...         ...   
 658      0.462798  1.324591    -0.063185    -0.435439  -0.762915   -0.804554   
 1170     0.387640  0.202932    -0.063185     0.454875   0.194299   -0.804554   
 684     -0.392793  1.240678     0.646183    -0.435439   0.878733    0.504983   
 869      0.536077  0.214412     0.646183    -0.435439   0.716427    1.296428   
 1361     1.807736  1.167758     0.646183 

In [793]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,8.710000e-02,1.720000e-02,1.292000e-01,8.964000e-01,0.0099,7.300000e-03,0.667
br,Bayesian Ridge,8.190000e-02,1.720000e-02,1.280000e-01,8.930000e-01,0.0098,6.800000e-03,0.099
lightgbm,Light Gradient Boosting Machine,9.370000e-02,1.920000e-02,1.373000e-01,8.833000e-01,0.0106,7.800000e-03,0.254
et,Extra Trees Regressor,9.350000e-02,2.000000e-02,1.399000e-01,8.777000e-01,0.0108,7.800000e-03,2.029
ridge,Ridge Regression,8.900000e-02,2.050000e-02,1.399000e-01,8.721000e-01,0.0107,7.400000e-03,0.031
rf,Random Forest Regressor,9.690000e-02,2.120000e-02,1.438000e-01,8.709000e-01,0.0111,8.100000e-03,2.045
huber,Huber Regressor,9.690000e-02,2.370000e-02,1.511000e-01,8.520000e-01,0.0117,8.100000e-03,0.247
omp,Orthogonal Matching Pursuit,9.340000e-02,2.560000e-02,1.530000e-01,8.408000e-01,0.0118,7.800000e-03,0.033
ada,AdaBoost Regressor,1.242000e-01,2.800000e-02,1.656000e-01,8.316000e-01,0.0127,1.030000e-02,0.428
knn,K Neighbors Regressor,1.195000e-01,2.910000e-02,1.688000e-01,8.226000e-01,0.0130,9.900000e-03,0.089


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=8696, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [794]:
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm
from lightgbm import LGBMRegressor 

In [795]:
baseline_model = GradientBoostingRegressor()
baseline_model.fit(df_train, log_target)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [796]:
# from sklearn.ensemble import RandomForestRegressor
# import pickle
# classifier = RandomForestRegressor()
# classifier.fit(x_train,y_train)
# filename = 'HousePred_XGB.pkl'
# pickle.dump(classifier,open(filename,'wb'))

In [799]:
br_params = {
    'n_iter': 304,
    'tol': 0.16864712769300896,
    'alpha_1': 5.589616542154059e-07,
    'alpha_2': 9.799343618469923,
    'lambda_1': 1.7735725582463822,
    'lambda_2': 3.616928181181732e-06
}

lightgbm_params = {
    'num_leaves': 39,
    'max_depth': 2,
    'learning_rate': 0.13705339989856127,
    'n_estimators': 273
}

ridge_params = {
    'alpha': 631.1412445239156
}

In [800]:
models = {'gbr':GradientBoostingRegressor(),
          'br':BayesianRidge(**br_params),
          'ridge':Ridge(**ridge_params),
          'lgbm':LGBMRegressor(**lightgbm_params),
          'hr':HuberRegressor()}


In [801]:
for name, model in models.items():
  model.fit(df_train, log_target)

#evaluation

In [802]:
results = {}

kf = KFold(n_splits=10)

for name, model in models.items():
    result = np.exp(np.sqrt(-cross_val_score(model, df_train, log_target, scoring='neg_mean_squared_error', cv=kf)))
    results[name] = result

In [803]:
for name, result in results.items():
    print("----------\n" + name)
    print(np.mean(result))
    print(np.std(result))

----------
gbr
1.134291801014157
0.020805754504807296
----------
br
1.1426832536443232
0.02597244546980759
----------
ridge
1.1431038725251788
0.02435182010041749
----------
lgbm
1.1320239167087887
0.018414464115058593
----------
hr
6.029157049945745
4.008522198540676


In [804]:
y_pred = (
    0.3 * np.exp(models['gbr'].predict(df_test)) +
    0.25 * np.exp(models['br'].predict(df_test)) +
    0.15 * np.exp(models['ridge'].predict(df_test))+
    0.3 * np.exp(models['lgbm'].predict(df_test)) )

#result


In [805]:
# y_pred = baseline_model.predict(df_test)

In [806]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('sample_submission.csv')
datasets = pd.concat([sub_df['Id'],pred],axis = 1)
datasets.columns = ['Id','SalePrice']
datasets.to_csv('submission.csv',index= False)